CLAHE & Median Filter Preprocessed Fine - Tuned MobileNetV2 Trained Model.ipynb

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths
train_dir = "/content/drive/MyDrive/Pneumonia Disease Detection CLAHE & Median Filter Preprocessed Dataset/Training Data"
val_dir = "/content/drive/MyDrive/Pneumonia Disease Detection CLAHE & Median Filter Preprocessed Dataset/Validation Data"
test_dir = "/content/drive/MyDrive/Pneumonia Disease Detection CLAHE & Median Filter Preprocessed Dataset/Testing Data"

# Image settings
img_size = (224, 224)
batch_size = 32
epochs = 25

# Data augmentation for training
train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_aug.flow_from_directory(
    train_dir,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Validation and test generators (no augmentation)
val_test_aug = ImageDataGenerator(rescale=1./255)

val_gen = val_test_aug.flow_from_directory(
    val_dir,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_gen = val_test_aug.flow_from_directory(
    test_dir,
    target_size=img_size,
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Get number of classes
num_classes = len(train_gen.class_indices)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights_dict = dict(enumerate(class_weights))

# Load base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze initial layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model with label smoothing
loss_fn = CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=Adam(learning_rate=0.0001), loss=loss_fn, metrics=['accuracy'])

# Checkpoint callback
checkpoint_path = '/content/drive/MyDrive/Pneumonia_MobileNetV2_Best_Model_Initial.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Initial training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[checkpoint],
    class_weight=class_weights_dict
)

# Unfreeze some layers for fine-tuning
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss=loss_fn, metrics=['accuracy'])

# Fine-tune training
checkpoint_path_finetuned = '/content/drive/MyDrive/Pneumonia_MobileNetV2_FineTuned_Model.h5'
fine_tune_checkpoint = ModelCheckpoint(checkpoint_path_finetuned, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,  # Optional: increase if needed
    callbacks=[fine_tune_checkpoint],
    class_weight=class_weights_dict
)

# Evaluate model on test data
loss, acc = model.evaluate(test_gen, verbose=1)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")

# Classification report
Y_pred = model.predict(test_gen)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_gen.classes
class_labels = list(test_gen.class_indices.keys())
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3477 images belonging to 3 classes.
Found 498 images belonging to 3 classes.
Found 993 images belonging to 3 classes.


<ipython-input-2-a590d6ef6705>:83: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_159']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 710ms/step - accuracy: 0.5057 - loss: 1.0448
Epoch 1: val_accuracy improved from -inf to 0.69880, saving model to /content/drive/MyDrive/Pneumonia_MobileNetV2_Best_Model_Initial.h5


...

109/109 ━━━━━━━━━━━━━━━━━━━━ 131s 773ms/step - accuracy: 0.6713 - loss: 0.8069 - val_accuracy: 0.7410 - val_loss: 0.6820
Epoch 3/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 702ms/step - accuracy: 0.7171 - loss: 0.7376
Epoch 3: val_accuracy improved from 0.74096 to 0.76707, saving model to /content/drive/MyDrive/Pneumonia_MobileNetV2_Best_Model_Initial.h5


109/109 ━━━━━━━━━━━━━━━━━━━━ 84s 771ms/step - accuracy: 0.7170 - loss: 0.7376 - val_accuracy: 0.7671 - val_loss: 0.6596
Epoch 4/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 711ms/step - accuracy: 0.7440 - loss: 0.7109
Epoch 4: val_accuracy improved from 0.76707 to 0.77912, saving model to /content/drive/MyDrive/Pneumonia_MobileNetV2_Best_Model_Initial.h5


109/109 ━━━━━━━━━━━━━━━━━━━━ 86s 790ms/step - accuracy: 0.7439 - loss: 0.7110 - val_accuracy: 0.7791 - val_loss: 0.6461
Epoch 5/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 711ms/step - accuracy: 0.7408 - loss: 0.7163
Epoch 5: val_accuracy improved from 0.77912 to 0.78514, saving model to /content/drive/MyDrive/Pneumonia_MobileNetV2_Best_Model_Initial.h5


109/109 ━━━━━━━━━━━━━━━━━━━━ 85s 781ms/step - accuracy: 0.7408 - loss: 0.7161 - val_accuracy: 0.7851 - val_loss: 0.6381
Epoch 6/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.7380 - loss: 0.7002
Epoch 6: val_accuracy did not improve from 0.78514
109/109 ━━━━━━━━━━━━━━━━━━━━ 85s 780ms/step - accuracy: 0.7381 - loss: 0.7002 - val_accuracy: 0.7751 - val_loss: 0.6523
Epoch 7/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 715ms/step - accuracy: 0.7544 - loss: 0.6873
Epoch 7: val_accuracy improved from 0.78514 to 0.78916, saving model to /content/drive/MyDrive/Pneumonia_MobileNetV2_Best_Model_Initial.h5


109/109 ━━━━━━━━━━━━━━━━━━━━ 85s 785ms/step - accuracy: 0.7544 - loss: 0.6873 - val_accuracy: 0.7892 - val_loss: 0.6297
Epoch 8/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 702ms/step - accuracy: 0.7505 - loss: 0.6876
Epoch 8: val_accuracy did not improve from 0.78916
109/109 ━━━━━━━━━━━━━━━━━━━━ 84s 770ms/step - accuracy: 0.7505 - loss: 0.6876 - val_accuracy: 0.7771 - val_loss: 0.6294
Epoch 9/25
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 709ms/step - accuracy: 0.7746 - loss: 0.6670
Epoch 9: val_accuracy did not improve from 0.78916
109/109 ━━━━━━━━━━━━━━━━━━━━ 85s 778ms/step - accuracy: 0.7745 - loss: 0.6670 - val_accuracy: 0.7831 - val_loss: 0.6486
Epoch 10/25
 94/109 ━━━━━━━━━━━━━━━━━━━━ 10s 709ms/step - accuracy: 0.7736 - loss: 0.6642